In [1]:
import torch
import torch.nn.functional as F
import torchvision
import torch.nn as nn
from torchvision import transforms as T
from utils import data, evaluate, models, train, plot
from torchsummary import summary
from torchvision.models import densenet169, DenseNet169_Weights

# Parameter Initialization

In [2]:
data_path = 'data/asl_alphabet_train'
data_path_test_1 = 'data/asl_alphabet_test'
data_path_test_2 = 'data/extra_test'
samples = 87000
split_factor = [0.8, 0.2, 0.0]
batch_size = 128
lr = 5e-4
epochs = 2
image_size = (100, 100)
l2_reg = 1e-4

# Get Train, Val and Test Split

In [3]:
torch.manual_seed(0)
train_trans = T.Compose([T.ToTensor(), T.Resize(image_size), T.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1), 
                         T.RandomRotation(degrees=(-5,5)), T.RandomHorizontalFlip(0.2), 
                         T.Normalize((0.5190, 0.4992, 0.5140), (0.2244, 0.2523, 0.2616))])
test_trans = T.Compose([T.ToTensor(), T.Resize(image_size), T.Normalize((0.5190, 0.4992, 0.5140), (0.2244, 0.2523, 0.2616))])

dataset = torchvision.datasets.ImageFolder(root=data_path, transform=train_trans)
test_dataset_1 = torchvision.datasets.ImageFolder(root=data_path_test_1, transform=test_trans)
test_dataset_2 = torchvision.datasets.ImageFolder(root=data_path_test_2, transform=test_trans)

train_loader, valid_loader = data.split_dataset(dataset, samples, split_factor, batch_size)
test_loader_1 = torch.utils.data.DataLoader(test_dataset_1, batch_size=batch_size, shuffle=False)
test_loader_2 = torch.utils.data.DataLoader(test_dataset_2, batch_size=batch_size, shuffle=False)

# Model

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device} device')

model = torchvision.models.densenet169(weights=DenseNet169_Weights.IMAGENET1K_V1)
model = model.to(device)
summary = nn.Sequential(model, nn.Softmax(dim=1))
print(summary)

Using cuda device
Sequential(
  (0): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-0

## 1) Training Phase

In [5]:
train_loss_list, train_accuracy_list, val_loss_list, val_accuracy_list = train.train_model(model, train_loader, valid_loader, 
                                                                                           lr, epochs, device, l2_reg)

Epoch 1 -> Loss = 0.0447 | Train Accuracy = 98.60% | valation Accuracy = 98.64%
Epoch 2 -> Loss = 0.0193 | Train Accuracy = 99.38% | valation Accuracy = 99.28%
----------------------------------------------------------------------------------------------------
Time taken to train: 2374.26s
Average time of each epoch: 1187.13s


## 2) Testing Phase

### i) Test Set 1 Accuracy

In [6]:
test_accuracy, y_true, y_pred = evaluate.evaluate_model(model, test_loader_1, device)

Test Accuracy = 100.0000%
F1 Score = 1.0000
Recall = 1.0000
Precision = 1.0000


### i) Test Set 2 Accuracy

In [7]:
test_accuracy, y_true, y_pred = evaluate.evaluate_model(model, test_loader_2, device)

Test Accuracy = 53.3333%
F1 Score = 0.5214
Recall = 0.5333
Precision = 0.5959
